## Gray-Scott Model

Represents the reaction and diffusion of two generic chemical species U and V.  The concentration of each in space is represented by u and v.

* Each chemical diffuses through space at its own rate
* U is added at a const feed rate into the system
* Two units of species V can turn a unit of U into V: $$2V + U \rightarrow 3V$$
* Const kill rate removing V

Represented by PDE for the concentrations of $u(x, y, t)$ and $v(x, y, t)$:

$$\frac{\partial u}{\partial t} = D_u \nabla ^2 u - uv^2 + F(1 - u)$$
$$\frac{\partial v}{\partial t} = D_v \nabla ^2 v + uv^2 - (F + k)v$$

\nabla ^2 is the Laplacian:

$$\nabla ^2 u = \frac{\partial ^2 u}{\partial x^2} + \frac{\partial ^2 u}{\partial y^2}$$

$D_u \nabla ^2 u$ and $D_v \nabla ^2 v$ correspond to the spatial diffusion of the concentrations.  $D_u$ and $D_v$ are the rates of diffusion.

$uv^2$ corresponds to the reaction which requires one unit of U and two of V.

$F(1 - u)$ is the feed rate and $-(F + k)v$ is the kill rate.  The feed rate replenishes U and the kill rate diminishes V.

### Problem setup

* Discretize reaction-diffusion using FTCS and assume $\Delta x = \Delta y = \delta$
* For the timestep set $\Delta t = \frac{9}{40} \frac{\delta ^2}{max(D_u, D_v)}$
* Use zero Neumann BC on all sides of the domain
* Use given ICs
    * Grid of pointswith dimension 192 by 192 points
    * Domain is 5m by 5m
    * Final time is 8000s

In [1]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [2]:
n = 192

Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 #Bacteria 1

dh = 5 / (n - 1)

T = 8000

dt = .9 * dh ** 2 / (4 * max(Du, Dv))

nt = int(T / dt)

In [13]:
uvinitial = numpy.load('./uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']